In [1]:
from drccp.cvx_drccp import CVaRRelaxation, RiskConstrainedProgram
from drccp.drccp_tools import (rkhs_radius_CV, x_constraints,
                               get_linear_objective)
from drccp_utils.distributions import GaussianDistribution
from drccp_utils.rkhs_utils import rkhs_func, compute_bootstrap_rkhs_radius, cholesky_decomposition
from drccp_utils.plot_utils import NEURIPS_RCPARAMS, LINE_WIDTH
from drccp_utils.sos_utils import build_sos_basis, get_monoms_list, parse_sym_to_cvx

from collections import deque

import argparse
import numpy as np
import cvxpy as cp
import sympy as sp
import dill as pickle
from pathlib import Path
np.set_printoptions(suppress=True)

In [2]:
dim = 3
risk_level = 0.1

mean = np.zeros(dim)
cov = np.zeros((dim, dim))
np.fill_diagonal(cov, np.arange(0.5, 0.5*(dim + 1), 0.5))
dist = GaussianDistribution(dim, mean=mean, std=cov)
alpha = risk_level
eps_lst = [0, 'bootstrap', 'rate']
n_samples = 10
supp_points = 0
test_xi = dist.sample(10000)

objective = get_linear_objective(dim)
np.random.seed(10)
X = dist.sample(n_samples)
print(X.shape)

(10, 3)


In [3]:
xi_sym = sp.symbols('xi1:%d'%(dim+1))
xi_arr = np.array([xi for xi in xi_sym])
x_sym = sp.symbols('x1:%d'%(dim+1))
x_arr = np.array([x for x in x_sym])
x_var = cp.Variable(dim, name='x', pos=True)
var_dict = {x_arr[i]: x_var[i] for i in range(dim)}

In [4]:
f_const = (-xi_arr**4 + 3*xi_arr**3 + 5*xi_arr**2) @ x_arr - 1
f_eval = sp.lambdify((x_sym, xi_sym), f_const)
d = 4
f = f_const.as_poly()
f_monoms = get_monoms_list(f)
print(f)
print(f.gens, f.monoms())
print(f_monoms)

Poly(-x1*xi1**4 + 3*x1*xi1**3 + 5*x1*xi1**2 - x2*xi2**4 + 3*x2*xi2**3 + 5*x2*xi2**2 - x3*xi3**4 + 3*x3*xi3**3 + 5*x3*xi3**2 - 1, x1, x2, x3, xi1, xi2, xi3, domain='ZZ')
(x1, x2, x3, xi1, xi2, xi3) [(1, 0, 0, 4, 0, 0), (1, 0, 0, 3, 0, 0), (1, 0, 0, 2, 0, 0), (0, 1, 0, 0, 4, 0), (0, 1, 0, 0, 3, 0), (0, 1, 0, 0, 2, 0), (0, 0, 1, 0, 0, 4), (0, 0, 1, 0, 0, 3), (0, 0, 1, 0, 0, 2), (0, 0, 0, 0, 0, 0)]
[x1*xi1**4, x1*xi1**3, x1*xi1**2, x2*xi2**4, x2*xi2**3, x2*xi2**2, x3*xi3**4, x3*xi3**3, x3*xi3**2, 1]


In [5]:
f_const

x1*(-xi1**4 + 3*xi1**3 + 5*xi1**2) + x2*(-xi2**4 + 3*xi2**3 + 5*xi2**2) + x3*(-xi3**4 + 3*xi3**3 + 5*xi3**2) - 1

In [6]:
X @ xi_arr
k_func = lambda xi1, xi2: (1 + (xi1 @ xi2)**int(d))
K = k_func(X, xi_arr)
gamma_cvx = cp.Variable(shape=(n_samples, 1), name='gamma')
gamma_sym = sp.MatrixSymbol('gamma', n_samples, 1)
var_dict[gamma_sym] = gamma_cvx
g = (K @ gamma_sym)[0]
g0_cvx = cp.Variable(name='g0')
g0_sym = sp.Symbol('g0')
var_dict[g0_sym] = g0_cvx
t_cvx = cp.Variable(name='t')
t_sym = sp.Symbol('t')
var_dict[t_sym] = t_cvx

In [7]:
sp_to_cp = {
    sp.exp: cp.exp,
    sp.log: cp.log
}

In [8]:
lhs1 = g0_sym + g - t_sym - f_const
lhs2 = g0_sym + g

In [9]:
def parse(expr):
    """
    Parse a polynomial expression in its appropriate base.
    """
    # We reached a leave in the expression Tree
    # expr.is_symbol is necessary for MatrixSymbol Elements
    if expr.is_Atom or expr.is_symbol:
        if expr.func.is_Symbol:  # This is a scalar symbol
            return var_dict[expr]
        elif expr.func.is_symbol:  # This is a Matrix symbol
            return var_dict[expr.symbol][expr.indices]
        elif expr.func.is_number:
            return float(expr)
        else:
            raise ValueError
    else:
        expressions = []
        for arg in expr.args:
            expressions.append(parse(arg))
        if expr.func.is_Add:
            cvx_expr = sum(expressions)
        elif expr.func.is_Mul:
            cvx_expr = np.prod(expressions)
        elif expr.func.is_Pow:
            assert len(expressions) == 2, 'There should only be 2 elements!'
            cvx_expr = expressions[0]**expressions[1]
        elif expr.func in sp_to_cp:
            assert len(expressions) == 1, "There should only be one expression by now!"
            cvx_expr = sp_to_cp[expr.func](expression[0])
        else:
            raise ValueError('The following expression {0} of type {1} is not supported.'.format(expr, expr.func))
        return cvx_expr
    
def parse_sym_to_cvx(polynomial, var_dict, basis):
    """
    Parse a sympy polynomial to a cvx expresion.

    Parameters
    ----------
    polynomial: sympy.Poly
    var_dict: dict
        Dictionary that contains mapping from cvxpy.Variable to sympy.Symbol
    basis: iteratable(sp.Symbol, ...)

    Returns
    -------
    poly_dict: dict
        keys -- monomials in sympy form
        values -- coefficients in cvxpy form
    m: list
        List of monomials in polynom
    """
    poly = polynomial.as_poly(basis)
#     print("polynomial: ", poly)
    monoms = get_monoms_list(poly)
    poly_dict = {}
    for m in monoms:
        c = poly.coeff_monomial(m)
        poly_dict[m] = parse(c)
#         print("Coefficient: ", c)
#         print("Prased CVX: ", poly_dict[m])
    return poly_dict, set(monoms)


In [10]:
def create_sos_constraints(lhs_constraint, var_sym, var_arr, var_dict, eps=1e-8):
    """
    Create a list with cvx constraints.
    
    Parameters
    ----------
    lhs_constraint: sympy.Poly
        The lefthandside constraint we want to get a SOS decomposition of.
    var_sym: iteratable(sympy.Symbol, ...)
        Iteratable containing all the sympy Variables.
    var_arr: np.ndarray
        Numpy array with all the variable for matrix mulitplication.
    var_dict: dict
        Dictionary mapping sympy Variables to cvxpy Variables
    
    Returns
    -------
    constraints: list
    """
    lhs_dict, lhs_monoms = parse_sym_to_cvx(lhs_constraint, var_dict, var_sym)
    
    # construct an appropriate SOS basis
    bsos, Q_sym, var_dict = build_sos_basis(lhs_monoms, var_arr, var_dict)
#     print("SOS basis: ", bsos)
#     print("Monomials to match: ", lhs_monoms)
    Q = sp.Matrix(Q_sym)
    n_sos = Q.shape[0]

    # SOS decomposition as the RHS of the constraint
    rhs = (bsos.transpose() @ Q @ bsos)[0, 0].as_poly(var_sym)
    rhs_dict, rhs_monoms = parse_sym_to_cvx(rhs, var_dict, var_sym)

    # Create cvxpy constraints
    constraints = [var_dict[Q_sym] >> eps * np.eye(n_sos)]
    for m in lhs_monoms:
        if m not in rhs_dict:
            continue
#         print('LHS constraint: ', lhs_dict[m])
#         print('RHS constraint: ', rhs_dict[m])
        constraints.append(lhs_dict[m] == rhs_dict[m])
    return constraints, Q_sym


In [11]:
def x_constraints(x, dim=None):
    return [
        x >= 0,
        cp.sum(x) <= 1
    ]

In [12]:
lhs1

g0 - t - x1*(-xi1**4 + 3*xi1**3 + 5*xi1**2) - x2*(-xi2**4 + 3*xi2**3 + 5*xi2**2) - x3*(-xi3**4 + 3*xi3**3 + 5*xi3**2) + (0.149041082381412*(-0.819488013595362*xi1 + xi2 - 0.016525805094112*xi3)**4 + 1)*gamma[1, 0] + (10.928149439765*(-0.769156951836942*xi1 - 0.593893581784757*xi2 + xi3)**4 + 1)*gamma[6, 0] + (7.07391859138153*(-0.670055810785976*xi1 + 0.438591736394606*xi2 + xi3)**4 + 1)*gamma[0, 0] + (0.0111819211070073*(0.00933164102242155*xi1 + 0.333806526295204*xi2 + xi3)**4 + 1)*gamma[2, 0] + (7.82692983522969*(0.0419156712786802*xi1 + xi2 + 0.822800922223252*xi3)**4 + 1)*gamma[8, 0] + (1.66891411357138*(0.0840718081882607*xi1 - xi2 + 0.47965770495835*xi3)**4 + 1)*gamma[5, 0] + (1.95168617834919*(0.136777911809514*xi1 + 0.869974235514892*xi2 - xi3)**4 + 1)*gamma[4, 0] + (8.59422465118613*(0.25324357558991*xi1 - 0.158421765883275*xi2 + xi3)**4 + 1)*gamma[9, 0] + (20.7846674011667*(0.789826437894916*xi1 + 0.124612155248982*xi2 - xi3)**4 + 1)*gamma[7, 0] + (0.523668542976538*(xi1 + 0

In [13]:
lhs2

g0 + (0.149041082381412*(-0.819488013595362*xi1 + xi2 - 0.016525805094112*xi3)**4 + 1)*gamma[1, 0] + (10.928149439765*(-0.769156951836942*xi1 - 0.593893581784757*xi2 + xi3)**4 + 1)*gamma[6, 0] + (7.07391859138153*(-0.670055810785976*xi1 + 0.438591736394606*xi2 + xi3)**4 + 1)*gamma[0, 0] + (0.0111819211070073*(0.00933164102242155*xi1 + 0.333806526295204*xi2 + xi3)**4 + 1)*gamma[2, 0] + (7.82692983522969*(0.0419156712786802*xi1 + xi2 + 0.822800922223252*xi3)**4 + 1)*gamma[8, 0] + (1.66891411357138*(0.0840718081882607*xi1 - xi2 + 0.47965770495835*xi3)**4 + 1)*gamma[5, 0] + (1.95168617834919*(0.136777911809514*xi1 + 0.869974235514892*xi2 - xi3)**4 + 1)*gamma[4, 0] + (8.59422465118613*(0.25324357558991*xi1 - 0.158421765883275*xi2 + xi3)**4 + 1)*gamma[9, 0] + (20.7846674011667*(0.789826437894916*xi1 + 0.124612155248982*xi2 - xi3)**4 + 1)*gamma[7, 0] + (0.523668542976538*(xi1 + 0.509037810483448*xi2 - 0.251377423795984*xi3)**4 + 1)*gamma[3, 0]

In [14]:
constraints = x_constraints(x_var)

In [15]:
const1, Q1 = create_sos_constraints(lhs1, xi_sym, xi_arr, var_dict)
constraints.extend(const1)

In [16]:
const2, Q2 = create_sos_constraints(lhs2, xi_sym, xi_arr, var_dict)
constraints.extend(const2)

In [17]:
kernel_matrix = k_func(X, X.T)
kernel_cholesky = cholesky_decomposition(kernel_matrix)
g_rkhs = kernel_matrix @ gamma_cvx
g_norm = cp.norm(kernel_cholesky @ gamma_cvx)
Eg_rkhs = cp.sum(g_rkhs) / n_samples

constraints.append(g0_cvx + Eg_rkhs + 0.1 * g_norm <= t_cvx*risk_level)

In [18]:
prob = cp.Problem(objective=cp.Maximize(objective(x_var)),
                 constraints=constraints)
prob.solve(solver="MOSEK")


0.15280606124558932

In [19]:
x_sol = x_var.value
g0 = g0_cvx.value
g_rkhs = g_rkhs.value
f_val = np.vstack([f_eval(x_sol, X[i, :]) for i in range(X.shape[0])])
t = t_cvx.value
q1 = var_dict[Q1].value
q2 = var_dict[Q2].value

In [20]:
print(g0 + g_rkhs)
print(g0 + g_rkhs - t - f_val)
print(np.linalg.eigvals(q1))
print(np.linalg.eigvals(q2))

[[0.00264337]
 [0.00070581]
 [0.00003257]
 [0.00113764]
 [0.00697782]
 [0.00481416]
 [0.00105179]
 [0.00196165]
 [0.00077545]
 [0.04288382]]
[[-0.488346  ]
 [ 0.92371488]
 [ 0.88505994]
 [ 0.90178238]
 [ 0.9151826 ]
 [ 0.79624816]
 [-0.79910531]
 [ 2.8741997 ]
 [-0.11907721]
 [-0.58110816]]
[0.00719968 0.00418946 0.00108948 0.00000001 0.00000001 0.00000001
 1.35776808 1.35776808 1.35776808 0.00000001]
[0.00719968 0.00418946 0.00108948 0.00000001 0.00000001 0.00000001
 1.35776808 1.35776808 1.35776808 0.00000001]
